In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pipeline import mini_algo, threshold_on_score, keep_valid_molecules, check_pains_brenk
from utils import evaluate_model

/Users/jackie16201/Desktop/Spring_2023/fragments_discovery_design_ML/src/pipeline.py:12: DeprecationWarning: The rdkit.Chem.MCS module is deprecated; please use rdkit.Chem.rdFMCS instead.
  from rdkit.Chem import MCS
/Users/jackie16201/opt/anaconda3/envs/chemprop/lib/python3.8/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/Users/jackie16201/opt/anaconda3/envs/chemprop/lib/python3.8/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


# Part 1: Get neural network baseline

First, we trained an underpowered model on the 2.3K compound staph screen. We use this model to predict on the 37K staph screen and the 27mil fragments. Training code can be found in 05_in_silico_algorithm_control.sh.

In [2]:
cpd_df = pd.read_csv('../out/controls/in_silico_algorithm_control_SA_37k_predictions_with_2300_model.csv')
all_cpd_df = cpd_df.copy()

# use the algo-predicted cpds as "positive" and the rest of cpds as negative - FOR ALL CPDS
all_cpd_df['predicted_with_gnn'] = [1.0 if x > 0.2 else 0.0 for x in list(all_cpd_df['class'])]
all_cpd_df['experimental_hit'] = list(all_cpd_df['Mean_50uM'] < 0.2)
print(str(sum(all_cpd_df['experimental_hit'])) + ' experimental hits out of ' + str(sum(all_cpd_df['predicted_with_gnn'])) + ' predicted hits')
evaluate_model(all_cpd_df, cutoff_for_positive = 0.2, actual_col='experimental_hit', predicted_col='predicted_with_gnn')

359 experimental hits out of 1518.0 predicted hits
precision recall: 0.08686796118642708
recall: 
0.13370473537604458
precision: 
0.03162055335968379


# Part 2: Get neural network baseline plus checking for PAINS and Brenk like fragment algorithm

In [3]:
print('length of cpd preds: ', len(cpd_df))
# threshold on score
cpd_df = threshold_on_score(cpd_df, 0.2, 'class')
# keep only valid molecules
cpd_df, cpd_mols = keep_valid_molecules(cpd_df, 'SMILES')
# keep only molecules without pains or brenk
cpd_df, cpd_mols = check_pains_brenk(cpd_df, cpd_mols)

predicted_positive_cpds = list(cpd_df['Compound_ID'])
all_cpd_df['predicted_with_gnn_plus_processing'] = [1.0 if x in predicted_positive_cpds else 0.0 for x in list(all_cpd_df['Compound_ID'])]
print(str(sum(all_cpd_df['experimental_hit'])) + ' experimental hits out of ' + str(sum(all_cpd_df['predicted_with_gnn_plus_processing'])) + ' predicted hits')
evaluate_model(all_cpd_df, cutoff_for_positive = 0.2, actual_col='experimental_hit', predicted_col='predicted_with_gnn_plus_processing')


length of cpd preds:  36977
length of df >0.2:  1518
length of df with valid mols:  1518
length of all preds with clean (no PAINS or Brenk) mols:  1351
359 experimental hits out of 1351.0 predicted hits
precision recall: 0.039841008255136576
recall: 
0.055710306406685235
precision: 
0.014803849000740192


# Part 3: Actual fragment algorithm test

In [4]:
# Values for processing fragments and compounds
fragment_path = '../out/controls/in_silico_algorithm_control_SA_all_combined_fragment_preds_gdb11_with_2300.csv'
compound_path = '../out/controls/in_silico_algorithm_control_SA_37k_predictions_with_2300_model.csv'
result_path = '../out/controls/in_silico_algorithm_control_SA_'
fragment_smi_col = 'SMILES'
compound_smi_col = 'SMILES'
fragment_hit_col = 'class'
compound_hit_col = 'class'

# filters and thresholds for fragments and compounds
fragment_score = 0.1
compound_score = 0.2
fragment_remove_pains_brenk = 'both' # one of 'both', 'pains', 'brenk', 'none'
compound_remove_pains_brenk = 'both' # one of 'both', 'pains', 'brenk', 'none'
fragment_require_more_than_coh = True
frags_cannot_disrupt_rings = True

rank_df, cpd_df = mini_algo(fragment_path=fragment_path,
                    compound_path=compound_path,
                    result_path=result_path,
                    fragment_smi_col=fragment_smi_col,
                    compound_smi_col=compound_smi_col,
                    fragment_hit_col=fragment_hit_col,
                    compound_hit_col=compound_hit_col,
                    fragment_score=fragment_score,
                    compound_score=compound_score,
                    fragment_remove_pains_brenk=fragment_remove_pains_brenk,
                    compound_remove_pains_brenk=compound_remove_pains_brenk, 
                    fragment_require_more_than_coh=fragment_require_more_than_coh,
                    frags_cannot_disrupt_rings=frags_cannot_disrupt_rings)

# save rank_df
rank_df = rank_df.sort_values('number_of_matched_molecules', ascending = False)

# take only the largest frags
rank_df = rank_df[rank_df['length_of_fragment'] > 7]
rank_df = rank_df[rank_df['number_of_matched_molecules'] < 400]

# get the final matching molecules for saving
all_matching_mol_indices = [x for l in list(rank_df['matched_molecules']) for x in l]

# deduplicate
all_matching_mol_indices = list(set(all_matching_mol_indices))
print('final number of molecules to test: ', len(all_matching_mol_indices))

# save the names
cpd_smiles = list(cpd_df['SMILES'])
all_matching_smis = [cpd_smiles[i] for i in list(set(all_matching_mol_indices))]

# and save the final molecules to df
all_matching_mols_df = pd.DataFrame()
all_matching_mols_df['SMILES'] = all_matching_smis
all_matching_mols_df

final_merge_df = all_matching_mols_df.merge(all_cpd_df, on = 'SMILES')
final_merge_df = final_merge_df.sort_values('class', ascending = False)
final_merge_df.to_csv('../out/controls/in_silico_algorithm_control_SA_final_proposed_fake_mols_to_order.csv', index = False)

predicted_positive_cpds = list(final_merge_df['Compound_ID'])
all_cpd_df['predicted_with_algo'] = [1.0 if x in predicted_positive_cpds else 0.0 for x in list(all_cpd_df['Compound_ID'])]
print(str(sum(all_cpd_df['experimental_hit'])) + ' experimental hits out of ' + str(sum(all_cpd_df['predicted_with_algo'])) + ' predicted hits')
evaluate_model(all_cpd_df, cutoff_for_positive = 0.2, actual_col='experimental_hit', predicted_col='predicted_with_algo')


Processing fragments...
length of df:  25020000
length of df >0.1:  3307974
length of df with more than C,O,H characters:  3202807
length of df with valid mols:  3202807
